In [36]:
from pynq import Overlay
Proc_RISCV = Overlay('/home/xilinx/pynq/overlays/design_RISCVEIRB/design_RISCVEIRB.bit')
Proc_RISCV?

In [37]:
RISCV = Proc_RISCV.My_RISCVEIRB_IP_0 
RISCV?


In [38]:
from pynq import MMIO
import numpy as np
import datetime
import os

class RISCVEIRB_Controller:

    
    def __init__(self, BASE_ADDRESS = 0x43C00000, ADDRESS_LENGTH = 0x64000):
        print("Création de la class RISCVEIRB_Controller")
        self.mmio = MMIO(BASE_ADDRESS, ADDRESS_LENGTH)


    ########## METHODES ##########
        
    ## WRITE ##

    def write_inst_mem_from_tab(self, mem_instruction, size = 32):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        CODE_RAM_SIZE = size
        print("\nInstruction Memory Write Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            # Envoie octets par octets
            for j in range (0, 4, 1):
                adresse = (4 * i) + j
                tableauOctets[j]  = (mem_instruction[i] >> (j*8)) % 256;
                data_ins = ((0 << 24) + (tableauOctets[j] << 16) + 0b0000000000001010); # Inst_Boot & Inst_RW_Boot
                # Ecriture dans la mémoire octets par octets
                self.mmio.write(0x04, adresse)
                self.mmio.write(0x00, int(data_ins))
            print("Octet value Write for current Instruction",hex(mem_instruction[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")

    def write_data_mem_from_tab(self, mem_data, size = 32):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        print("\nData Memory Write Process\n\r")
        CODE_RAM_SIZE = size
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                temp = (j  * 8)
                adresse = (4 * i) + j
                tableauOctets[j]  = (mem_data[i]>> temp)%256;
                data_val = (tableauOctets[j] << 24) + ((0 << 16) +  + 0b0000000000010100);
                self.mmio.write(0x08, adresse)    
                self.mmio.write(0x00, int(data_val))
            print("Octet value Write for current Instruction",hex(mem_data[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")
    

    ## READ ##

    def read_inst_mem(self, size = 32, log_opt = False, file_name = "inst_mem"):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        mem_instruction = np.zeros(size)
        CODE_RAM_SIZE = size
        #Instruction Memory Read Process  
        print("\nInstruction Memory Read Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                # Sequence lecture
                # Inst_RW_Boot <= 0 (READ)
                # Inst_Boot <= 1 
                self.mmio.write(0x00,0b00000000000000000000000000000010)
                adresse = ((4 * i) + j)
                self.mmio.write(0x04, adresse)
                tableauOctets[j] = self.mmio.read(0xC)>>16
                data_ins_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
            if (data_ins_rd < 0) : 
                data_ins_rd = (data_ins_rd + (1<<32))
            print("La valeur de l'instruction", i," est :", hex(data_ins_rd))
            mem_instruction[i] = data_ins_rd
         # Enregistrement dans un fichier log    
        if (log_opt):
            now = datetime.datetime.now()
            time = now.strftime("%Y-%m-%d_%H-%M-%S-%f")  # Ajout de %f pour les microsecondes
            log_file_name = "./log/"+time+"_"+file_name+".hex"
            with open(log_file_name, 'w') as f:
                for value in mem_instruction:
                    f.write(f'0x{int(value):08X},\n')
                   
            

    
    def read_data_mem(self, size = 32, log_opt = False, file_name = "data_mem"):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        mem_data = np.zeros(size)
        CODE_RAM_SIZE = size
        #Data Memory Read Process  
        print("\nInstruction Memory Read Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                # Data_RW_Boot <= 0 (READ)
                # Data_Boot <= 1 
                self.mmio.write(0x00,0b00000000000000000000000000000100)
                adresse = ((4 * i) + j)
                self.mmio.write(0x08, adresse)
                tableauOctets[j] = self.mmio.read(0xC)>>24
            data_val_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
            if (data_val_rd  < 0) : 
                data_val_rd  = (data_val_rd  + (1<<32))
            mem_data[i] = int(data_val_rd)
            print("La valeur de la donnée", i," est :", hex(data_val_rd))
        # Enregistrement dans un fichier log    
        if (log_opt):
            now = datetime.datetime.now()
            time_extension = now.strftime("%Y-%m-%d_%H-%M-%S-%f")  # Ajout de %f pour les microsecondes
            filename = "./log/"+time_extension+"_"+file_name+".hex"
            with open(filename, 'w') as f:
                for value in mem_data:
                    f.write(f'0x{int(value):08X},\n')

    def cpu_run(self):
        print("CPU RUN\n")

    def cpu_execution(self):
        self.mmio.write(0x0,0b00000000000000000000000000100000) # BOOT <= 1
        self.mmio.write(0x0,0b00000000000000000000000000000000) # BOOT <= 0
        for i in range(0, 500, 1):
            #Mode_debug  pour activé le CE : reg0(6)=1 & reg0(0)=1 => (0b00000000000000000000000001000001)
            #Mode_normal pour activé Le CE : reg0(6)=0 & reg0(0)=1 => (0b00000000000000000000000000000001)
            self.mmio.write(0x0,0b0000000000000000000000001000001)
            self.mmio.write(0x0,0b0000000000000000000000001000000)
            PC = self.mmio.read(0x10) # Sig_Adr_Inst_out
            Current_Ins = self.mmio.read(0x14) # Sig_Val_Out_Inst_out
            NextAdr_Ins = self.mmio.read(0x18) # Sig_New_Adr_Inst_out
            print("Program Counter (PC) value : ", PC, "(",int(PC/4),"), Current Instruction :", hex(Current_Ins) ,", New Address Instruction :", NextAdr_Ins,"\r")
            Mem_Data_Adr_Value = self.mmio.read(0x1C)
            Data_in  = self.mmio.read(0x20) # Sig_Val_In_Data_out
            Data_out = self.mmio.read(0x24) # Sig_Val_Out_Data_out
            print("Mem Data Address value : ", int(Mem_Data_Adr_Value) ,", Data In : ",hex(Data_in),", Data Out:",hex(Data_out)," \r");
            UAL_op = self.mmio.read(0x34)
            print("UAL operation: ",bin(UAL_op),"->",int(UAL_op),"\r");
            FSM_value = self.mmio.read(0x38)
            match int(FSM_value):
                case 0:
                    print("Current State Machine is : Init\r")
                case 1:
                    print("Current State Machine is : FetchIns\r")
                case 2: 
                    print("Current State Machine is : Decode\r")
                case 3: 
                    print("Current State Machine is : ExeAddr\r")
                case 4:
                    print("Current State Machine is : ExeOp\r")
                case 5:
                    print("Current State Machine is : ExeOpimm \r")
                case 6:
                    print("Current State Machine is : ExeLoad \r")
                case 7:
                    print("Current State Machine is : ExeWrite\r")
                case 8:
                    print("Current State Machine is : ExeCtrr\r")
                case 9:
                    print("Current State Machine is : ExeJal\r")
                case 10:
                    print("Current State Machine is : ExeJal2\r")
                case 11:
                    print("Current State Machine is : ExeJalr\r")
                case 12:
                    print("Current State Machine is : Undefined\r\n\n")
                case 13:
                    print("Current State Machine is : ExeLui\r")
                case 14:
                    print("Current State Machine is : ExeAuipc\r")
                case 15:
                    print("Current State Machine is : ExeNop\r")
                case _ :
                    print("Current State Machine is : Undefined\r\n\n")
            Date_UT_value = self.mmio.read(0x3C)
            print("Data in UT for Load Instruction",hex(Date_UT_value),",",bin(Date_UT_value),"\r\n")
            


    
    
## Fonctions utilitaires
def charger_fichier(path):
    if path.endswith('.hex'):
        with open(path, 'r') as file:
            data = np.array([int(line.strip().rstrip(','), 16) for line in file])
    else:
        print("Format de fichier non pris en charge.")
        return None
    return data

In [39]:
Controller = RISCVEIRB_Controller()

Création de la class RISCVEIRB_Controller


In [40]:
mem_inst = charger_fichier(path="riscveirb_benchmark_1.hex")
Controller.write_inst_mem_from_tab(mem_inst, size=5)


Instruction Memory Write Process

Octet value Write for current Instruction 0x200293 at address 0 : [ 0x0 ; 0x20 ; 0x2 ; 0x93 ]
Octet value Write for current Instruction 0x1e00313 at address 1 : [ 0x1 ; 0xe0 ; 0x3 ; 0x13 ]
Octet value Write for current Instruction 0x6283b3 at address 2 : [ 0x0 ; 0x62 ; 0x83 ; 0xb3 ]
Octet value Write for current Instruction 0x702423 at address 3 : [ 0x0 ; 0x70 ; 0x24 ; 0x23 ]
Octet value Write for current Instruction 0x8067 at address 4 : [ 0x0 ; 0x0 ; 0x80 ; 0x67 ]


In [41]:
Controller.cpu_execution()

Program Counter (PC) value :  0 ( 0 ), Current Instruction : 0x200293 , New Address Instruction : 4 
Mem Data Address value :  2 , Data In :  0x0 , Data Out: 0x0  
UAL operation:  0b1 -> 1 
Current State Machine is : FetchIns
Data in UT for Load Instruction 0x0 , 0b0 

Program Counter (PC) value :  0 ( 0 ), Current Instruction : 0x200293 , New Address Instruction : 4 
Mem Data Address value :  2 , Data In :  0x0 , Data Out: 0x0  
UAL operation:  0b1 -> 1 
Current State Machine is : Decode
Data in UT for Load Instruction 0x0 , 0b0 

Program Counter (PC) value :  0 ( 0 ), Current Instruction : 0x200293 , New Address Instruction : 4 
Mem Data Address value :  2 , Data In :  0x0 , Data Out: 0x0  
UAL operation:  0b1 -> 1 
Current State Machine is : ExeOpimm 
Data in UT for Load Instruction 0x0 , 0b0 

Program Counter (PC) value :  4 ( 1 ), Current Instruction : 0x1e00313 , New Address Instruction : 8 
Mem Data Address value :  30 , Data In :  0x0 , Data Out: 0x0  
UAL operation:  0b1 -> 1 

In [42]:
Controller.read_data_mem(size=32, log_opt=True, file_name="mem_data_bench1")


Instruction Memory Read Process

La valeur de la donnée 0  est : 0x0
La valeur de la donnée 1  est : 0x0
La valeur de la donnée 2  est : 0x20
La valeur de la donnée 3  est : 0x0
La valeur de la donnée 4  est : 0x0
La valeur de la donnée 5  est : 0x0
La valeur de la donnée 6  est : 0x0
La valeur de la donnée 7  est : 0x0
La valeur de la donnée 8  est : 0x0
La valeur de la donnée 9  est : 0x0
La valeur de la donnée 10  est : 0x0
La valeur de la donnée 11  est : 0x0
La valeur de la donnée 12  est : 0x0
La valeur de la donnée 13  est : 0x0
La valeur de la donnée 14  est : 0x0
La valeur de la donnée 15  est : 0x0
La valeur de la donnée 16  est : 0x0
La valeur de la donnée 17  est : 0x0
La valeur de la donnée 18  est : 0x0
La valeur de la donnée 19  est : 0x0
La valeur de la donnée 20  est : 0x0
La valeur de la donnée 21  est : 0x0
La valeur de la donnée 22  est : 0x0
La valeur de la donnée 23  est : 0x0
La valeur de la donnée 24  est : 0x0
La valeur de la donnée 25  est : 0x0
La valeur de 